매일경제(https://www.mk.co.kr; **부동산, 증권 category 제외)**의 모든 기사 관련 데이터를 수집 
정제한 후 jsonl file에 저장하는 코드를 작성   

In [1]:
import requests
from bs4 import BeautifulSoup
import json 
import re
homepage ='https://www.mk.co.kr'


In [2]:
response = requests.get(homepage)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
results = soup.find_all('a')

In [4]:
#정규표현식으로 뉴스기사 url 찾기
pattern = re.compile(r".+news\/[a-zA-Z]+\/[0-9]+")

In [5]:
link_collection =[] #url 모을 리스트

In [6]:
for result in results:
    link = result.get('href')
    if pattern.match(link):
#         print(link)
        link_collection.append(link)

In [7]:
len(link_collection)

159

# link_collection 정제하기 

In [8]:
# url에서 제외 해야할 키워드(부동산, 증권 category, notice)
exclude_words = ["realestate", "stock","notice", 'culture']#여행#공지

#제외 해야할 url 모음
exclude_links=[]

In [9]:
for word in exclude_words: #키워드 제거할 링크 찾기 
    for link in link_collection:
        if word in link:
            #print(link)
            exclude_links.append(link)
            
for exclude in exclude_links: #링크 제거 
    link_collection.remove(exclude)

In [10]:
print("가져올 기사 개수:", len(link_collection))
link_collection[:3]

가져올 기사 개수: 138


['/news/politics/11076304',
 '/news/politics/11076307',
 '/news/politics/11076117']

In [11]:
for i in range(len(link_collection)):
    if homepage not in link_collection[i]: #완전한 url이면 아니라면
        link_collection[i] = homepage + link_collection[i]

#  제목, 날짜, 소제목, 내용 가져오는 함수


In [12]:
# 제목 받는 함수 
def get_title(soup):
    title = None
    title = soup.find('h2', class_ = "news_ttl")
    if title:
        title = title.text.strip()
#         print('제목:',title)
    return title

#날짜 받는 함수 
def get_date(soup):
    date = None 
    date = soup.find('dl', class_='registration')
    date = date.find('dd').text.split()[0]    
#     print('날짜:', date)
    
    return date

#소제목, 본문 가져오는 함수 
def get_text(soup): 
    content = soup.find('div', class_='news_cnt_detail_wrap') 
    heading = None #소제목
    maintext = None #본문 
    
    #### 소제목 가져온 후 제거 #######
    subcontent = content.find('div', class_='mid_title')
    if subcontent:
        heading = subcontent.text.strip() 
        subcontent.decompose() #본문에서 소제목 제거 (클리닝)
#     print('소제목:', subheading)
    
    ########## 본문 글 가져오기##################
    figures = content.find_all('div', class_='thumb_area img')
    for f in figures:
        f.decompose() # 중간 사진과 부가설명 제거 
       
    maintext = content.text.replace('\n', '').strip()
#     print("본문: \n", main_text)
    
    return heading, maintext

# 크롤링 멈춤 방지 에러받기 

In [13]:
##main
article_data =[]

for link in link_collection[:]:
    try:
        article = requests.get(link)
        soup = BeautifulSoup(article.content, 'html.parser')
    
        heading, maintext = get_text(soup)
        
        if len(maintext)<100:#본문이 비어있는 형식의 뉴스
            print("\n-no content:", link) #저장하지 않기
        else: #저장해주기
            news_article={
                'title': get_title(soup),
                'date' : get_date(soup),
                'heading': heading,
                'main_text' : maintext
                }
            article_data.append(news_article) 
            
    except: #요청한 페이지를 찾을 수 없을 때 
        print('\n-error occurred: ', link)

print('THE END')


-error occurred:  https://www.mk.co.kr/news/world/11071728

-error occurred:  https://www.mk.co.kr/news/world/11071727

-no content: https://www.mk.co.kr/news/columnists/11073889

-no content: https://www.mk.co.kr/news/photos/11075770

-no content: https://www.mk.co.kr/news/photos/11075735

-no content: https://www.mk.co.kr/news/sports/11075235

-no content: https://www.mk.co.kr/news/sports/11075234
THE END


In [14]:
len(article_data)

131

In [15]:
'''
파일 쓰기
'''

with open('article_data.jsonl', encoding='utf-8', mode ='w') as file:
    for article in article_data[:]:
        file.write(json.dumps(article, ensure_ascii=False)+'\n')

In [16]:
'''
#파일읽기(검토용)

data_list=[]
with open('article_data.jsonl', "r", encoding='utf-8') as file:
    for line in file:
        data_list.append(json.loads(line))
print(len(data_list))


for data in data_list[:3]:
    print(data)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$')
'''

'\n#파일읽기(검토용)\n\ndata_list=[]\nwith open(\'article_data.jsonl\', "r", encoding=\'utf-8\') as file:\n    for line in file:\n        data_list.append(json.loads(line))\nprint(len(data_list))\n\n\nfor data in data_list[:3]:\n    print(data)\n    print(\'$$$$$$$$$$$$$$$$$$$$$$$$$$\')\n'